In [2]:
import requests
import time
import csv
import http.client
import json
import pandas as pd
import numpy as np

from geopy.exc import GeocoderTimedOut
from geopy.geocoders import Nominatim

import plotly.express as px
import plotly.graph_objects as go

import country_converter as coco

# import geopandas as gpd

# Tutorials used
- Insomnia scraping  https://www.youtube.com/watch?v=DqtlR0y0suo
- geopy https://www.geeksforgeeks.org/how-to-find-longitude-and-latitude-for-a-list-of-regions-or-country-using-python/
- Country converter https://pypi.org/project/country-converter/

# Run next cell to scrape BCL site and put results into .csv

In [3]:
conn = http.client.HTTPSConnection("www.bcliquorstores.com")

payload = ""

ALC_TYPES = ['wine', 'beer', 'spirits', 'coolers/ciders']
# ALC_TYPES = ['beer']

results = []

for ALC in ALC_TYPES:
    headers = {
    'Cookie': "_fbp=fb.1.1658289464792.1891546519; _sp_id.671f=dd097fb6-7220-426a-881a-85903387b7f6.1658289465.1.1658289545.1658289465.09e4a09f-1391-48d7-93eb-1cd4fd68c587; _sp_ses.671f=*",
    'Accept': "application/json, text/plain, */*",
    'Accept-Encoding': "gzip, deflate, br",
    'Host': "www.bcliquorstores.com",
    'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/15.5 Safari/605.1.15",
    'Accept-Language': "en-gb",
    'Referer': f"https://www.bcliquorstores.com/product-catalogue?category={ALC}&sort=name.raw:asc&page=1",
    'Connection': "keep-alive"
    }
    query_string = f"/ajax/browse?category={ALC}&sort=name.raw%3Aasc&size=1&page=1&="
    conn.request("GET", query_string, payload, headers)
    res = conn.getresponse()
    data = res.read()

    my_json = data.decode('utf8').replace("'", '"')
    num_alc = json.loads(my_json)['hits']['total']

    query_string = f"/ajax/browse?category={ALC}&sort=name.raw%3Aasc&size={num_alc}&page=1&="
    conn.request("GET", query_string, payload, headers)

    res = conn.getresponse()
    data = res.read()
    my_json = data.decode('utf8').replace("'", '"')
    json_str = json.loads(my_json)  
    for p in json_str['hits']['hits']:
        results.append(p)

df = pd.json_normalize(results)
df.to_csv('firstresult.csv')

## Run to load info from .csv

In [4]:
df = pd.read_csv('firstresult.csv')
df

,Unnamed: 0,_index,_type,_id,_score,sort,_source.redVarietal,_source.isVQA,_source.subCategory.description,_source.subCategory.id,...,_source.promotions,_source.nameSanitized,_source.isBCVQA,_source._currentPrice,_source._promotionEndDate,_source.votes,_source.category.description,_source.category.id,_source.is_special_release,_source.availability_override
0,0,bcls,product,122867,0.601447,['ABRUZZO PECORINO - BARONE DI VALFORTE 2020'],NaN,NaN,Italy Wine,1043,...,NaN,abruzzo pecorino - barone di valforte 2020,NaN,17.99,2022-12-31T00:00:00.000-08:00,84,Wine,1005,NaN,NaN
1,1,bcls,product,432123,0.601447,['ABRUZZO PINOT GRIGIO - MARE DI SIRENA'],NaN,NaN,Italy Wine,1043,...,"[{'urlAlias': '/promotion/new-arrivals', 'name...",abruzzo pinot grigio - mare di sirena,NaN,12.99,2022-12-31T00:00:00.000-08:00,18,Wine,1005,NaN,NaN
2,2,bcls,product,712331,0.601447,['ACQUESI - ASTI'],NaN,NaN,Italy Wine,1043,...,NaN,acquesi - asti,NaN,18.49,2022-12-31T00:00:00.000-08:00,293,Wine,1005,NaN,NaN
3,3,bcls,product,783498,0.595873,['AGLIANICO DEL VULTURE - BASILISCO TEODOSIO 2...,Aglianico,NaN,Italy Wine,1043,...,NaN,aglianico del vulture - basilisco teodosio 2019,NaN,21.99,2022-12-31T00:00:00.000-08:00,233,Wine,1005,NaN,NaN
4,4,bcls,product,556126,0.601447,['AIX EN PROVENCE ROSE - BIELER PERE & FILS CU...,NaN,NaN,France Wine,1037,...,NaN,aix en provence rose - bieler pere & fils cuve...,NaN,15.97,2022-12-31T00:00:00.000-08:00,116,Wine,1005,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5543,5543,bcls,product,59587,0.200588,['WHITE CLAW - WATERMELON CAN'],NaN,NaN,Coolers,1015,...,NaN,white claw - watermelon can,NaN,14.49,NaN,71,Coolers & Ciders,1003,NaN,NaN
5544,5544,bcls,product,512049,0.200588,['WHITE CLAW ICED TEA - LEMON CAN'],NaN,NaN,Coolers,1015,...,NaN,white claw iced tea - lemon can,NaN,11.97,2022-12-31T00:00:00.000-08:00,40,Coolers & Ciders,1003,NaN,NaN
5545,5545,bcls,product,477180,0.203842,['WHITE CLAW ICED TEA - VARIETY PACK'],NaN,NaN,Coolers,1015,...,NaN,white claw iced tea - variety pack,NaN,27.99,NaN,30,Coolers & Ciders,1003,NaN,NaN
5546,5546,bcls,product,444128,0.200588,['WHITE CLAW SURGE - VARIETY PACK 12 CAN'],NaN,NaN,Coolers,1015,...,"[{'urlAlias': '/promotion/new-arrivals', 'name...",white claw surge - variety pack 12 can,NaN,29.99,NaN,10,Coolers & Ciders,1003,NaN,NaN


In [5]:
countries = df['_source.countryName'].unique()
num_countries = len(countries)

## Assigning a latitude and longitude to the unique countries

In [6]:
longitude = []
latitude = []


def findGeocode(city):
    # try and catch is used to overcome
    # the exception thrown by geolocator
    # using geocodertimedout  
    try:
          
        # Specify the user_agent as your
        # app name it should not be none
        geolocator = Nominatim(user_agent="johann@pdrayne.com")
          
        return geolocator.geocode(city)
      
    except GeocoderTimedOut:
          
        return findGeocode(city)  

count = 1
for i in (countries):
      
    if findGeocode(i) != None:
           
        loc = findGeocode(i)
        # coordinates returned from 
        # function is stored into
        # two separate list
        latitude.append(loc.latitude)
        longitude.append(loc.longitude)
       
    # if coordinate for a city not
    # found, insert "NaN" indicating 
    # missing value 
    else:
        latitude.append(np.nan)
        longitude.append(np.nan)
    
    print(f'Logging {i}, {count}/{num_countries}')
    count += 1

Logging Italy, 1/65
Logging France, 2/65
Logging Spain, 3/65
Logging Portugal, 4/65
Logging Greece, 5/65
Logging Canada, 6/65
Logging Argentina, 7/65
Logging USA, 8/65
Logging Japan, 9/65
Logging Australia, 10/65
Logging Luxembourg, 11/65
Logging Germany, 12/65
Logging South Africa, 13/65
Logging Korea - South, 14/65
Logging Austria, 15/65
Logging Uruguay, 16/65
Logging Chile, 17/65
Logging China, 18/65
Logging Israel, 19/65
Logging New Zealand, 20/65
Logging Lebanon, 21/65
Logging Croatia, 22/65
Logging Turkey, 23/65
Logging United Kingdom, 24/65
Logging Hungary, 25/65
Logging Georgia, 26/65
Logging Bulgaria, 27/65
Logging Switzerland, 28/65
Logging Montenegro, 29/65
Logging Morocco, 30/65
Logging Czech Republic, 31/65
Logging Netherlands, 32/65
Logging Iceland, 33/65
Logging Trinidad And Tobago, 34/65
Logging Ireland, 35/65
Logging Denmark, 36/65
Logging Mexico, 37/65
Logging Belgium, 38/65
Logging Philippines, 39/65
Logging Thailand, 40/65
Logging Singapore, 41/65
Logging Poland, 42

## Assigning a country code

In [7]:
standard_names02 = coco.convert(names=countries, to='ISO2') 
standard_names03 = coco.convert(names=countries, to='ISO3') 

country_df = pd.DataFrame({
    'country':countries,
    'lat':latitude,
    'lon':longitude,
    'country_code02':standard_names02,
    'country_code03':standard_names03
})
print(country_df)

Virgin Islands not found in regex
Virgin Islands not found in regex


      country        lat         lon country_code02 country_code03
0       Italy  42.638426   12.674297             IT            ITA
1      France  46.603354    1.888334             FR            FRA
2       Spain  39.326068   -4.837979             ES            ESP
3    Portugal  39.662165   -8.135352             PT            PRT
4      Greece  38.995368   21.987713             GR            GRC
..        ...        ...         ...            ...            ...
60    Vietnam  15.926666  107.965086             VN            VNM
61   Ukraine   49.487197   31.271832             UA            UKR
62       Fiji -18.123970  179.012274             FJ            FJI
63  Guatemala  15.585555  -90.345759             GT            GTM
64     Latvia  56.840649   24.753764             LV            LVA

[65 rows x 5 columns]


## Adding latitiude, longitude and country code to large dataframe

In [8]:
lat_mat = pd.DataFrame(df.apply (lambda row: country_df['lat'].loc[country_df['country'] == row['_source.countryName']], axis=1))
lon_mat = pd.DataFrame(df.apply (lambda row: country_df['lon'].loc[country_df['country'] == row['_source.countryName']], axis=1))
coco_mat02 = pd.DataFrame(df.apply (lambda row: country_df['country_code02'].loc[country_df['country'] == row['_source.countryName']], axis=1))
coco_mat03 = pd.DataFrame(df.apply (lambda row: country_df['country_code03'].loc[country_df['country'] == row['_source.countryName']], axis=1))


temp_lat = lat_mat
temp_lon = lon_mat
temp_coco02 = coco_mat02
temp_coco03 = coco_mat03
for index in range(int(np.shape(lat_mat)[1])):
    temp_lat = pd.DataFrame(temp_lat[0].fillna(lat_mat[index]))
    temp_lon = pd.DataFrame(temp_lon[0].fillna(lon_mat[index]))
    temp_coco02 = pd.DataFrame(temp_coco02[0].fillna(coco_mat02[index]))
    temp_coco03 = pd.DataFrame(temp_coco03[0].fillna(coco_mat03[index]))

df['lat'] = temp_lat
df['lon'] = temp_lon
df['country_code02'] = temp_coco02
df['country_code03'] = temp_coco03

## Counting alcohol per country

In [9]:
num_p_country = np.zeros(len(country_df['country'].index))

for index, country in enumerate(country_df['country']):
    num = len(df.loc[df['_source.countryName'] == country].index)
    num_p_country[index] = num

country_df['num_p_country'] = num_p_country

## Plotting interactive graph to visualise where BCL products come from

https://plotly.com/python/map-configuration/

In [11]:
fig = go.Figure(data=go.Choropleth(
    locations = country_df['country_code03'],
    z = country_df['num_p_country'],
    colorscale = 'Blues',
    autocolorscale=False,
    reversescale=False,
    marker_line_color='darkgray',
    marker_line_width=0.5,
    colorbar_tickprefix = '',
    colorbar_title = 'No. Products',
))

fig.update_layout(
    title_text='BCL products from country',
    geo=dict(
        showframe=True,
        showcoastlines=True,
        projection_type='equirectangular'
    ),
    annotations = [dict(
        x=0.55,
        y=0.1,
        xref='paper',
        yref='paper',
        text='Source: <a href="https://www.cia.gov/library/publications/the-world-factbook/fields/2195.html">\
            CIA World Factbook</a>',
        showarrow = False
    )]
)


fig.show(renderer="notebook_connected")

## Useful Links for plotting these graphs

https://towardsdatascience.com/work-with-geospatial-data-and-create-interactive-maps-using-geopy-and-plotly-28178d2868f1 </br>
https://plotly.com/python/choropleth-maps/ </br>
https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv </br>
https://www.infragistics.com/help/reveal/location-data-requirements </br>